![sql](img/sql-logo.jpg)

# A beginner's guide to databases, SQL, & using them with `pandas`

**Scenario:** You are a data analyst for the Homeland Security, trying to create reports on the active airports world wide. The data you need to access is in a SQL database. YOu need to be able to query for the data in a database!

## Learning goals:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use `JOIN` to merge tables along logical columns
    - Grouping Data with SQL, inlcuding `HAVING`
- Goal 5: Convert SQL to pandas

## Goal 1: Summarize

To date the data we've seen has looked like [this.](https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Intakes/wter-evkm)

That is how we need data to look to run analysis and build models.<br>
But it doesn't _live_ there in it's native state.

[Netflix has a great article](https://medium.com/netflix-techblog/notebook-innovation-591ee3221233) describing three different data roles at their company, their different needs, and their toolsets.

![netflix](img/netflix-data-roles.jpeg)

Examining that graphic, SQL shows up as one of the tools of the _Data Engineer_ 

Data Engineers provide the essential data architecture services that make data science possible.

![hierarchy](img/ai-hierachy.png)

[Source: Monica Rogati’s fantastic Medium post “The AI Hierarchy of Needs”
](https://hackernoon.com/the-ai-hierarchy-of-needs-18f111fcc007)

## What does it mean to **Engineer Data**?

Let's start with a basic scenario:<br>
You are HR. You have *no* current database. 
How would you set it up?

Data needs considerations:
- What you want to store
- What "views" you anticipate wanting in the future

Structure considerations:
- Speed of retrieval
- How much data you are accessing
- How much you are storing

![etl](img/etl.png)

[img source: Jeff Hammerbacher’s slide from UC Berkeley CS 194 course ](https://bcourses.berkeley.edu/courses/1377158/pages/cs-194-16-introduction-to-data-science-fall-2015)

### What is a Relational Database? 

![rdb](img/relational-dbms-model.png)
[reference for image ](https://www.studytonight.com/dbms/database-model.php)

### POPULAR RDBMS

- SQLite
- MySQL
- PostgreSql
- Oracle DB
- SQL Server

***
### SQL Readings

[History of SQL](https://www.businessnewsdaily.com/5804-what-is-sql.html)

[The original SQL paper from the 1970s.](https://www.seas.upenn.edu/~zives/03f/cis550/codd.pdf)

***
## Goal 2: Database terminology

### Relational Database Schema

![schema](img/MySQL_Schema_Music_Example.png)

[source of image](https://database.guide/what-is-a-database-schema/)

***
### SQLite

<img src="img/SQLite-Python.jpg" width=500>

- C library (fast)
- lightweight disk-based database
- that doesn’t require a separate server process
- nonstandard variant of the SQL query language
- applications can use SQLite for internal data storage
- use case: prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle

[sqlite documentation](https://docs.python.org/2/library/sqlite3.html)



## But what about connecting to database servers?

To connect to other database types there are many different libraries:
- Oracle: [cx_Oracle](https://oracle.github.io/python-cx_Oracle/)
- MySQL: [MySQL-python](https://github.com/farcepest/MySQLdb1)
- PostgreSQL: [Psycopg2](http://initd.org/psycopg/docs/)
- Microsoft SQL Server: [pymssql](http://www.pymssql.org/en/stable/)

These all implement [PEP 249: DB API v2](https://www.python.org/dev/peps/pep-0249/)

## Goal 3: Get going with sqlite!

In [1]:
import sqlite3
import pandas as pd

#### Load a DB object with `connect` and `cursor`

In [2]:
con = sqlite3.connect('data/flights.db')
cursor = con.cursor()

#### Use sqlite_master to find all the tables in the schema
Get the schema of a database from a db in sqlite

In [3]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('airports',), ('airlines',), ('routes',)]


In [4]:
cursor.description

(('name', None, None, None, None, None, None),)

In [5]:
cursor.execute("SELECT name, type FROM sqlite_master;")
print(cursor.fetchall())

[('airports', 'table'), ('ix_airports_index', 'index'), ('airlines', 'table'), ('ix_airlines_index', 'index'), ('routes', 'table'), ('ix_routes_index', 'index')]


In [6]:
cursor.execute("SELECT * FROM sqlite_master;")
results = cursor.fetchall()
for res in results:
    print(res)

('table', 'airports', 'airports', 2, 'CREATE TABLE airports (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [city] TEXT,\n  [country] TEXT,\n  [code] TEXT,\n  [icao] TEXT,\n  [latitude] TEXT,\n  [longitude] TEXT,\n  [altitude] TEXT,\n  [offset] TEXT,\n  [dst] TEXT,\n  [timezone] TEXT\n)')
('index', 'ix_airports_index', 'airports', 3, 'CREATE INDEX ix_airports_index ON airports ([index])')
('table', 'airlines', 'airlines', 945, 'CREATE TABLE airlines (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [alias] TEXT,\n  [iata] TEXT,\n  [icao] TEXT,\n  [callsign] TEXT,\n  [country] TEXT,\n  [active] TEXT\n)')
('index', 'ix_airlines_index', 'airlines', 946, 'CREATE INDEX ix_airlines_index ON airlines ([index])')
('table', 'routes', 'routes', 1393, 'CREATE TABLE routes (\n[index] INTEGER,\n  [airline] TEXT,\n  [airline_id] TEXT,\n  [source] TEXT,\n  [source_id] TEXT,\n  [dest] TEXT,\n  [dest_id] TEXT,\n  [codeshare] TEXT,\n  [stops] TEXT,\n  [equipment] TEXT\n)')
('index', 'ix_routes_i

#### Get information about one table

**A note about** `execute`<br>
Each time you use it, you reset the value of cursor

In [7]:
cursor.execute("SELECT * FROM airports as a").fetchone()

(0,
 '1',
 'Goroka',
 'Goroka',
 'Papua New Guinea',
 'GKA',
 'AYGA',
 '-6.081689',
 '145.391881',
 '5282',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [8]:
cursor.fetchone()

(1,
 '2',
 'Madang',
 'Madang',
 'Papua New Guinea',
 'MAG',
 'AYMD',
 '-5.207083',
 '145.7887',
 '20',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [9]:
cursor.fetchmany(4)

[(2,
  '3',
  'Mount Hagen',
  'Mount Hagen',
  'Papua New Guinea',
  'HGU',
  'AYMH',
  '-5.826789',
  '144.295861',
  '5388',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (3,
  '4',
  'Nadzab',
  'Nadzab',
  'Papua New Guinea',
  'LAE',
  'AYNZ',
  '-6.569828',
  '146.726242',
  '239',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (4,
  '5',
  'Port Moresby Jacksons Intl',
  'Port Moresby',
  'Papua New Guinea',
  'POM',
  'AYPY',
  '-9.443383',
  '147.22005',
  '146',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (5,
  '6',
  'Wewak Intl',
  'Wewak',
  'Papua New Guinea',
  'WWK',
  'AYWK',
  '-3.583828',
  '143.669186',
  '19',
  '10',
  'U',
  'Pacific/Port_Moresby')]

#### Use description

In [10]:
cursor.description

(('index', None, None, None, None, None, None),
 ('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('code', None, None, None, None, None, None),
 ('icao', None, None, None, None, None, None),
 ('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None),
 ('altitude', None, None, None, None, None, None),
 ('offset', None, None, None, None, None, None),
 ('dst', None, None, None, None, None, None),
 ('timezone', None, None, None, None, None, None))

#### Or use `Pragma`
`Pragma` tool [link here](https://www.sqlite.org/pragma.html#pragma_table_info)

**output**<br>
`(column id, column name, data type, whether or not the column can be NULL, and the default value for the column)`


*N.B. Pragma tools are unique to SQLite, and cannot be used in other SQL dialects*

In [11]:
cursor.execute("PRAGMA table_info(airports)")
info = cursor.fetchall()
print(info)
print()
print(*info, sep = "\n") # '*' is Python for 'unpack'

[(0, 'index', 'INTEGER', 0, None, 0), (1, 'id', 'TEXT', 0, None, 0), (2, 'name', 'TEXT', 0, None, 0), (3, 'city', 'TEXT', 0, None, 0), (4, 'country', 'TEXT', 0, None, 0), (5, 'code', 'TEXT', 0, None, 0), (6, 'icao', 'TEXT', 0, None, 0), (7, 'latitude', 'TEXT', 0, None, 0), (8, 'longitude', 'TEXT', 0, None, 0), (9, 'altitude', 'TEXT', 0, None, 0), (10, 'offset', 'TEXT', 0, None, 0), (11, 'dst', 'TEXT', 0, None, 0), (12, 'timezone', 'TEXT', 0, None, 0)]

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'city', 'TEXT', 0, None, 0)
(4, 'country', 'TEXT', 0, None, 0)
(5, 'code', 'TEXT', 0, None, 0)
(6, 'icao', 'TEXT', 0, None, 0)
(7, 'latitude', 'TEXT', 0, None, 0)
(8, 'longitude', 'TEXT', 0, None, 0)
(9, 'altitude', 'TEXT', 0, None, 0)
(10, 'offset', 'TEXT', 0, None, 0)
(11, 'dst', 'TEXT', 0, None, 0)
(12, 'timezone', 'TEXT', 0, None, 0)


In [12]:
# Example of using '*' for unpacking

z = (1,2)
print('z:', z)

x,y = z
print('x:', x)
print('y:', y)

print(z[0],z[1])
print(*z)

z: (1, 2)
x: 1
y: 2
1 2
1 2


#### Making fetch happen

`.fetchall()` is how you get the query results out of the object.

You can also `.fetchone()` or `.fetchmany()`

**Task:** Get the descriptive data for airlines and routes tables

In [13]:
cursor.execute("PRAGMA table_info(airlines)")
info = cursor.fetchall()
print(*info, sep = "\n")

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'alias', 'TEXT', 0, None, 0)
(4, 'iata', 'TEXT', 0, None, 0)
(5, 'icao', 'TEXT', 0, None, 0)
(6, 'callsign', 'TEXT', 0, None, 0)
(7, 'country', 'TEXT', 0, None, 0)
(8, 'active', 'TEXT', 0, None, 0)


In [14]:
cursor.execute("PRAGMA table_info(routes)")
info = cursor.fetchall()
print(*info, sep = "\n")

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'airline', 'TEXT', 0, None, 0)
(2, 'airline_id', 'TEXT', 0, None, 0)
(3, 'source', 'TEXT', 0, None, 0)
(4, 'source_id', 'TEXT', 0, None, 0)
(5, 'dest', 'TEXT', 0, None, 0)
(6, 'dest_id', 'TEXT', 0, None, 0)
(7, 'codeshare', 'TEXT', 0, None, 0)
(8, 'stops', 'TEXT', 0, None, 0)
(9, 'equipment', 'TEXT', 0, None, 0)


***
## Goal 4: Use basic SQL commands 

### The Structure of a SQL Query

<img src='img/sql_statement.jpg'/>

### Constructing SQL queries

**`SELECT`**:  The columns you want
  - options: `DISTINCT`, using `AS` to rename columns, single number aggregates

**`FROM`:** the source tables
  - options: also uses aliasing with `AS`; joining with `[LEFT|INNER|RIGHT|FULL] JOIN ___ [ON|USING]`

**`WHERE`**: your filters
  - options: comparators like `=` & `>=`; `BETWEEN`, `IN`, `LIKE` (with wildcards `%`); booleans like `AND`, `OR`, `NOT`

**`ORDER BY`**: sorting
  - options: `ASC` (default) and `DESC`

**`LIMIT`**:  # of rows to return (pair with `OFFSET`)

**Task**: 
- Select only active airlines in the UK from the airlines table
- Select the unique list of countries with airports

In [15]:
cursor.execute("""
    SELECT name FROM airlines WHERE country='United Kingdom' AND active='Y' ORDER BY lower(name)
""")
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]
df

# Best SQL practices: Single equals sign for equivalence. Use single quotes.

,name
0,AD Aviation
1,Air Cudlua
2,Air Foyle
3,Air Southwest
4,Air Wales
5,All Europe
6,Astraeus
7,Aurigny Air Services
8,BA CityFlyer
9,BBN-Airways


In [16]:
cursor.execute("""
    SELECT DISTINCT country, count(code) FROM airports GROUP BY country ORDER BY country
""")
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]
df

,country,count(code)
0,Afghanistan,16
1,Albania,1
2,Algeria,35
3,American Samoa,3
4,Angola,22
...,...,...
235,West Bank,0
236,Western Sahara,3
237,Yemen,10
238,Zambia,8


### SQL Joins

SQL joins can be used to both **add** data to a table and **remove** data from a table. 

**Task** Write a query that will join the latitude and longitude data from the airports table to the information on the routes table

In [17]:
cursor.execute("""
    SELECT source, sources.latitude AS source_lat, sources.longitude AS source_long, 
           dest, dests.latitude AS dest_lat, dests.longitude AS dest_long
    FROM routes
    JOIN airports AS sources ON sources.id = source_id
    JOIN airports AS dests ON dests.id = dest_id
""")
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]
df

,source,source_lat,source_long,dest,dest_lat,dest_long
0,AER,43.449928,39.956589,KZN,55.606186,49.278728
1,ASF,46.283333,48.006278,KZN,55.606186,49.278728
2,ASF,46.283333,48.006278,MRV,44.225072,43.081889
3,CEK,55.305836,61.503333,KZN,55.606186,49.278728
4,CEK,55.305836,61.503333,OVB,55.012622,82.650656
...,...,...,...,...,...,...
67197,WYA,-33.0589,137.514,ADL,-34.945,138.530556
67198,DME,55.408611,37.906111,FRU,43.061306,74.477556
67199,FRU,43.061306,74.477556,DME,55.408611,37.906111
67200,FRU,43.061306,74.477556,OSS,40.608989,72.793269


### Grouping statements

Combines `select` and `group by` when you want aggregates by values

`select` `min(x)` ... `max()`, `sum()`, etc

`group by x`

**Task**<br>
- Which countries have the highest amount of active airlines?
- Which countries have the highest amount of inactive airlines?
- What about airports by timezones?

In [18]:
# Which countries have the highest amount of active airports?
cursor.execute("""
-- your code here;
""")
print(*cursor.fetchall(), sep='\n')

In [19]:
# Which countries have the highest amount of inactive airports?
cursor.execute("""
-- your code here;
""")
print(*cursor.fetchall(), sep='\n')

In [20]:
# What about airports by timezones?
cursor.execute("""
-- your code here;""")
print(*cursor.fetchall(), sep='\n')

In [21]:
# It's always a good idea to close our connections when we're done
cursor.close()
con.close()

## Goal 5: Transfering from sqlite to pandas

In [22]:
pd_con = sqlite3.connect("data/flights.db")
df = pd.read_sql_query("SELECT * FROM airports LIMIT 5;", pd_con)
df

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby


**Task**: 
Convert one of the earlier queries in the lesson to a pandas data frame

In [23]:
# Which countries have the highest amount of active airports?
sql = '''
    SELECT source, sources.latitude AS source_lat, sources.longitude AS source_long, 
       dest, dests.latitude AS dest_lat, dests.longitude AS dest_long
    FROM routes
    JOIN airports AS sources ON sources.id = source_id
    JOIN airports AS dests ON dests.id = dest_id
'''
pd.read_sql_query(sql, pd_con)

,source,source_lat,source_long,dest,dest_lat,dest_long
0,AER,43.449928,39.956589,KZN,55.606186,49.278728
1,ASF,46.283333,48.006278,KZN,55.606186,49.278728
2,ASF,46.283333,48.006278,MRV,44.225072,43.081889
3,CEK,55.305836,61.503333,KZN,55.606186,49.278728
4,CEK,55.305836,61.503333,OVB,55.012622,82.650656
...,...,...,...,...,...,...
67197,WYA,-33.0589,137.514,ADL,-34.945,138.530556
67198,DME,55.408611,37.906111,FRU,43.061306,74.477556
67199,FRU,43.061306,74.477556,DME,55.408611,37.906111
67200,FRU,43.061306,74.477556,OSS,40.608989,72.793269


In [24]:
pd_con.close()

## Bonus: SQL-like querying in Pandas

`.query()`

[query documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html)

In [25]:
# Get data for an example
import pandas as pd
shelter_data=pd.read_csv('https://data.austintexas.gov/api/views/9t4d-g238/rows.csv?accessType=DOWNLOAD')

In [26]:
max_data = shelter_data.query('Name == "Max"')
max_data.head()


,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
50,A736717,Max,10/21/2016 02:45:00 PM,10/21/2016 02:45:00 PM,10/16/2013,Return to Owner,NaN,Dog,Neutered Male,3 years,Pug Mix,Brown Brindle
160,A766116,Max,02/13/2018 11:14:00 AM,02/13/2018 11:14:00 AM,02/02/2015,Return to Owner,NaN,Dog,Neutered Male,3 years,Rottweiler/Staffordshire,Blue/Tan
379,A688196,Max,09/16/2014 12:20:00 PM,09/16/2014 12:20:00 PM,09/16/2005,Euthanasia,Suffering,Dog,Neutered Male,9 years,Jack Russell Terrier Mix,Chocolate/White
501,A579608,Max,07/21/2014 12:43:00 PM,07/21/2014 12:43:00 PM,06/09/2010,Return to Owner,NaN,Dog,Neutered Male,4 years,Labrador Retriever Mix,Black/White
808,A745151,Max,03/13/2017 07:12:00 PM,03/13/2017 07:12:00 PM,11/13/2016,Return to Owner,NaN,Dog,Neutered Male,3 months,German Shepherd Mix,White


In [27]:
#vs
shelter_data[shelter_data['Name']=="Max"].head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
50,A736717,Max,10/21/2016 02:45:00 PM,10/21/2016 02:45:00 PM,10/16/2013,Return to Owner,NaN,Dog,Neutered Male,3 years,Pug Mix,Brown Brindle
160,A766116,Max,02/13/2018 11:14:00 AM,02/13/2018 11:14:00 AM,02/02/2015,Return to Owner,NaN,Dog,Neutered Male,3 years,Rottweiler/Staffordshire,Blue/Tan
379,A688196,Max,09/16/2014 12:20:00 PM,09/16/2014 12:20:00 PM,09/16/2005,Euthanasia,Suffering,Dog,Neutered Male,9 years,Jack Russell Terrier Mix,Chocolate/White
501,A579608,Max,07/21/2014 12:43:00 PM,07/21/2014 12:43:00 PM,06/09/2010,Return to Owner,NaN,Dog,Neutered Male,4 years,Labrador Retriever Mix,Black/White
808,A745151,Max,03/13/2017 07:12:00 PM,03/13/2017 07:12:00 PM,11/13/2016,Return to Owner,NaN,Dog,Neutered Male,3 months,German Shepherd Mix,White


In [28]:
shelter_data.query('`Animal Type` == "Dog"').head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
1,A776359,Gizmo,07/18/2018 04:02:00 PM,07/18/2018 04:02:00 PM,07/12/2017,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown
2,A720371,Moose,02/13/2016 05:59:00 PM,02/13/2016 05:59:00 PM,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
14,A765349,Einstein,06/08/2018 01:04:00 PM,06/08/2018 01:04:00 PM,01/18/2009,Adoption,Foster,Dog,Neutered Male,9 years,Chihuahua Shorthair Mix,Tricolor
15,A760697,Star,10/26/2017 03:22:00 PM,10/26/2017 03:22:00 PM,10/23/2007,Transfer,Partner,Dog,Intact Male,10 years,Yorkshire Terrier Mix,Brown/Black
16,A767231,Millie,02/25/2018 05:19:00 PM,02/25/2018 05:19:00 PM,02/25/2017,Return to Owner,NaN,Dog,Spayed Female,1 year,Jack Russell Terrier/Chihuahua Shorthair,White/Tan
